##### Table 1: genotypes at each of the five focal SNPs in Vgsc

In [1]:
from collections import defaultdict
import dask.array as da
import numpy as np
import pandas as pd

import allel

import ingenos

In [2]:
base_path = "/overflow/dschridelab/users/rrlove/aedes/"
md_path = f"{base_path}metadata/"
results_path = f"{base_path}results/"
zarr_path = "/proj/dschridelab/rrlove/aedes/vcf/filtered_110122/"

In [3]:
chroms = ["AaegL5_1", "AaegL5_2", "AaegL5_3"]
countries = ["Brazil", "Colombia", "USA", "Kenya", "Senegal", "Gabon"]

##### read in the genotypes for AaegL5_3

In [4]:
def return_data(zarr_path, data_type, data_name):
    
    fetch_str = f"{data_type}/{data_name}" 
    
    return da.from_zarr(zarr_path, component=fetch_str)

In [5]:
chroms_dict = defaultdict(dict)

chroms = ["AaegL5_3"]

for chrom in chroms:
    
    temp_in_path = zarr_path + chrom

    chroms_dict[chrom]["pos"] = return_data(temp_in_path, "variants", "POS")
    chroms_dict[chrom]["chrom"] = return_data(temp_in_path, "variants", "CHROM")
    chroms_dict[chrom]["ref"] = return_data(temp_in_path, "variants", "REF")
    chroms_dict[chrom]["alt"] = return_data(temp_in_path, "variants", "ALT")
    chroms_dict[chrom]["ac"] = return_data(temp_in_path, "variants", "AC")
    chroms_dict[chrom]["qd"] = return_data(temp_in_path, "variants", "QD")
    chroms_dict[chrom]["mq"] = return_data(temp_in_path, "variants", "MQ")
    chroms_dict[chrom]["fs"] = return_data(temp_in_path, "variants", "FS")
    chroms_dict[chrom]["mqrs"] = return_data(temp_in_path, "variants", "MQRankSum")
    chroms_dict[chrom]["rprs"] = return_data(temp_in_path, "variants", "ReadPosRankSum")
    chroms_dict[chrom]["sor"] = return_data(temp_in_path, "variants", "SOR")
    chroms_dict[chrom]["is_snp"] = return_data(temp_in_path, "variants", "is_snp")
    chroms_dict[chrom]["numalt"] = return_data(temp_in_path, "variants", "numalt")
    chroms_dict[chrom]["filter_pass"] = return_data(temp_in_path, "variants", "FILTER_PASS")

    chroms_dict[chrom]["gt"] = return_data(temp_in_path, "calldata", "GT")
    chroms_dict[chrom]["gq"] = return_data(temp_in_path, "calldata", "GQ")
    chroms_dict[chrom]["ad"] = return_data(temp_in_path, "calldata", "AD")
    chroms_dict[chrom]["pl"] = return_data(temp_in_path, "calldata", "PL")
    chroms_dict[chrom]["dp"] = return_data(temp_in_path, "calldata", "DP")
    
    print(chrom, chroms_dict[chrom]["gt"].shape)
    
samples = da.from_zarr(temp_in_path, component="samples")

AaegL5_3 (27660249, 131, 2)


##### read in and filter the metadata

In [6]:
md = pd.read_table(
    md_path + "whole_sample_sorted_country.031522.csv", 
    sep="\t",)

md["locality"] = md["location"].str.split(": ", expand=True)[1]

md.head()

,sample_id,sample_short,location,sex,batch,sample_id_cat,country,locality
0,FEMALE_1-F1_CGCATGAT-TCAGGCTT_S1,FEMALE_1,Colombia: Rio Claro,F,1,FEMALE_1-F1_CGCATGAT-TCAGGCTT_S1,Colombia,Rio Claro
1,FEMALE_10-F10_GTGCCATA-ACTAGGAG_S2,FEMALE_10,Colombia: Rio Claro,F,1,FEMALE_10-F10_GTGCCATA-ACTAGGAG_S2,Colombia,Rio Claro
2,FEMALE_11-F11_CGTTGCAA-CGCTCTAT_S3,FEMALE_11,Colombia: Rio Claro,F,1,FEMALE_11-F11_CGTTGCAA-CGCTCTAT_S3,Colombia,Rio Claro
3,FEMALE_12-F12_TGAAGACG-TGGCATGT_S4,FEMALE_12,Colombia: Rio Claro,F,1,FEMALE_12-F12_TGAAGACG-TGGCATGT_S4,Colombia,Rio Claro
4,FEMALE_14-F14_ACGTTCAG-GCACAACT_S6,FEMALE_14,Colombia: Rio Claro,F,1,FEMALE_14-F14_ACGTTCAG-GCACAACT_S6,Colombia,Rio Claro


##### remove close kin

In [7]:
to_drop = np.loadtxt(md_path + "close_kin_removed_new_dataset.txt", dtype="str")

to_drop

array(['SRR11006849', 'SRR11006846', 'SRR6768018', 'FEMALE_17',
       'FEMALE_22', 'SRR11006674', 'SRR11006683', 'SRR11006754'],
      dtype='<U11')

In [8]:
to_drop_bool = (~(md["sample_short"].isin(to_drop)).values)

np.sum(to_drop_bool)

123

In [9]:
md_filtered = md.loc[to_drop_bool]

md_filtered.shape

(123, 8)

In [10]:
countries = md_filtered["country"].unique()

country_bools = {}

for country in countries:
    
    country_bools[country] = (md_filtered["country"] == country).values
    
country_bools.keys()

dict_keys(['Colombia', 'Kenya', 'Senegal', 'Gabon', 'Brazil', 'USA'])

In [11]:
[(key, np.sum(country_bools[key])) for key in country_bools.keys()]

[('Colombia', 32),
 ('Kenya', 17),
 ('Senegal', 19),
 ('Gabon', 13),
 ('Brazil', 16),
 ('USA', 26)]

##### focal SNPs: 
315939224 (not called in this dataset)

315983763

315999297

316014588

316080722

In [12]:
focal_snps = [315939224, 315983763, 315999297, 316014588, 316080722]
pos = chroms_dict[chrom]["pos"].compute()

In [13]:
focal_snps_dict = {}

for snp in focal_snps:
    
    focal_snps_dict[snp] = (pos == snp)
    
    print(snp, np.sum(focal_snps_dict[snp]))

315939224 0
315983763 1
315999297 1
316014588 1
316080722 1


In [14]:
for country, country_flt in country_bools.items():
    
    print(country)
    
    gt = allel.GenotypeArray(chroms_dict[chrom]["gt"])\
    .subset(sel1 = to_drop_bool).subset(
        sel1 = country_flt)

    gq = chroms_dict[chrom]["gq"].compute()[:, to_drop_bool][:, country_flt]
    
    gt.mask = gq < 20
    
    for snp in focal_snps:
    
        pos_flt = focal_snps_dict[snp]
        
        print(snp)
        
        if np.sum(pos_flt) > 0:

            print("homozygous ref: ", gt.subset(sel0 = pos_flt).count_hom_ref())
            print("het: ", gt.subset(sel0 = pos_flt).count_het())
            print("homozygous alt: ", gt.subset(sel0 = pos_flt).count_hom_alt())
            print("missing: ", np.sum(gt.subset(sel0 = pos_flt).is_missing()), "\n")


Colombia
315939224
315983763
homozygous ref:  6
het:  16
homozygous alt:  9
missing:  1 

315999297
homozygous ref:  6
het:  15
homozygous alt:  9
missing:  2 

316014588
homozygous ref:  4
het:  14
homozygous alt:  8
missing:  6 

316080722
homozygous ref:  6
het:  16
homozygous alt:  10
missing:  0 

Kenya
315939224
315983763
homozygous ref:  17
het:  0
homozygous alt:  0
missing:  0 

315999297
homozygous ref:  4
het:  10
homozygous alt:  2
missing:  1 

316014588
homozygous ref:  14
het:  0
homozygous alt:  0
missing:  3 

316080722
homozygous ref:  16
het:  0
homozygous alt:  0
missing:  1 

Senegal
315939224
315983763
homozygous ref:  18
het:  0
homozygous alt:  0
missing:  1 

315999297
homozygous ref:  8
het:  9
homozygous alt:  0
missing:  2 

316014588
homozygous ref:  18
het:  0
homozygous alt:  0
missing:  1 

316080722
homozygous ref:  19
het:  0
homozygous alt:  0
missing:  0 

Gabon
315939224
315983763
homozygous ref:  12
het:  0
homozygous alt:  0
missing:  1 

31599929

##### read in the mpileup data to re-calculate genotypes for F1534C (315939224)

In [15]:
vcf = allel.read_vcf(
    "/proj/dschridelab/rrlove/aedes/vcf/Vssc/AaegL5_3_Vssc_mpileup.vcf.gz", 
                     fields=["*"])

vcf.keys()

/nas/longleaf/home/rrlove/.conda/envs/rrlove/lib/python3.8/site-packages/allel/io/vcf_read.py:1732: UserWarning: invalid INFO header: '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">\n'
  warnings.warn('invalid INFO header: %r' % header)


dict_keys(['samples', 'calldata/GP', 'calldata/GQ', 'calldata/GT', 'calldata/PL', 'variants/AC', 'variants/ALT', 'variants/AN', 'variants/BQB', 'variants/CHROM', 'variants/DP', 'variants/DP4', 'variants/FILTER_PASS', 'variants/HOB', 'variants/ICB', 'variants/ID', 'variants/IDV', 'variants/IMF', 'variants/INDEL', 'variants/MQ', 'variants/MQ0F', 'variants/MQB', 'variants/MQSB', 'variants/POS', 'variants/QUAL', 'variants/REF', 'variants/RPB', 'variants/SGB', 'variants/altlen', 'variants/is_snp', 'variants/numalt'])

In [16]:
md_vcf = md.copy()

md_vcf["sample_id_cat"] = md_vcf["sample_id"].astype("category")

md_vcf["sample_id_cat"].cat.set_categories(pd.Series(vcf["samples"]), 
                                   inplace=True)

md_vcf.sort_values("sample_id_cat", inplace=True)

md_vcf.head()

,sample_id,sample_short,location,sex,batch,sample_id_cat,country,locality
96,SRR11006847,SRR11006847,Brazil: Santarem,NaN,PRJNA602495,SRR11006847,Brazil,Santarem
82,SRR11006830,SRR11006830,Gabon: Franceville,NaN,PRJNA602495,SRR11006830,Gabon,Franceville
35,SRR11006666,SRR11006666,Kenya: KayaBomu,NaN,PRJNA602495,SRR11006666,Kenya,KayaBomu
76,SRR11006824,SRR11006824,Gabon: Franceville,NaN,PRJNA602495,SRR11006824,Gabon,Franceville
80,SRR11006828,SRR11006828,Gabon: Franceville,NaN,PRJNA602495,SRR11006828,Gabon,Franceville


In [17]:
np.sum(~(md_vcf["sample_id"] == vcf["samples"]))

0

In [18]:
gt = allel.GenotypeArray(vcf["calldata/GT"][vcf["variants/is_snp"]])

gt_2 = allel.GenotypeArray(vcf["calldata/GT"][vcf["variants/is_snp"]])

gt.shape

(347281, 131, 2)

In [19]:
gq = vcf["calldata/GQ"][vcf["variants/is_snp"]]

pos = allel.SortedIndex(vcf["variants/POS"][vcf["variants/is_snp"]])

##### ID specimens to drop

In [20]:
to_drop_bool_vcf = (~(md_vcf["sample_short"].isin(to_drop)).values)

np.sum(to_drop_bool_vcf)

123

In [21]:
md_filtered_vcf = md_vcf.loc[to_drop_bool_vcf]

md_filtered_vcf.shape

(123, 8)

In [22]:
country_bools_vcf = {}

for country in countries:
    
    country_bools_vcf[country] = (md_filtered_vcf["country"] == country).values
    
country_bools_vcf.keys()

dict_keys(['Colombia', 'Kenya', 'Senegal', 'Gabon', 'Brazil', 'USA'])

In [23]:
pos_flt = (pos == 315939224)

np.sum(pos_flt)

1

In [24]:
for country, country_flt in country_bools_vcf.items():
    
    print(country)
    
    gt_chunk = \
    gt.subset(sel0 = pos_flt, sel1 = to_drop_bool_vcf).subset(sel1 = country_flt)

    gq_chunk = gq[pos_flt, :][:, to_drop_bool_vcf][:, country_flt]
    
    print("homozygous ref: ", gt_chunk.count_hom_ref())
    print("het: ", gt_chunk.count_het())
    print("homozygous alt: ", gt_chunk.count_hom_alt())
    print("missing: ", np.sum(gt_chunk.is_missing()), "\n")
    
    ##then, mask
    
    gt_chunk.mask = gq_chunk < 20
    
    print("homozygous ref, masked: ", gt_chunk.count_hom_ref())
    print("het, masked: ", gt_chunk.count_het())
    print("homozygous alt, masked: ", gt_chunk.count_hom_alt())
    print("missing, masked: ", np.sum(gt_chunk.is_missing()), "\n")


Colombia
homozygous ref:  0
het:  0
homozygous alt:  32
missing:  0 

homozygous ref, masked:  0
het, masked:  0
homozygous alt, masked:  19
missing, masked:  13 

Kenya
homozygous ref:  15
het:  0
homozygous alt:  2
missing:  0 

homozygous ref, masked:  0
het, masked:  0
homozygous alt, masked:  0
missing, masked:  17 

Senegal
homozygous ref:  19
het:  0
homozygous alt:  0
missing:  0 

homozygous ref, masked:  7
het, masked:  0
homozygous alt, masked:  0
missing, masked:  12 

Gabon
homozygous ref:  13
het:  0
homozygous alt:  0
missing:  0 

homozygous ref, masked:  2
het, masked:  0
homozygous alt, masked:  0
missing, masked:  11 

Brazil
homozygous ref:  0
het:  0
homozygous alt:  16
missing:  0 

homozygous ref, masked:  0
het, masked:  0
homozygous alt, masked:  1
missing, masked:  15 

USA
homozygous ref:  3
het:  0
homozygous alt:  23
missing:  0 

homozygous ref, masked:  0
het, masked:  0
homozygous alt, masked:  3
missing, masked:  23 

